In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-03-16 13:02:28.945251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 13:02:28.945364: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 13:02:29.067967: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/sample_submission.csv
/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/X_test.csv
/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/y_train.csv
/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/X_train.csv


In [2]:
xtrain=pd.read_csv("/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/X_train.csv")
xtrain.head()

,date,home_team,away_team,tournament,city,country,neutral
0,2001-03-11,Botswana,Mozambique,Friendly,Maseru,Lesotho,True
1,1982-06-22,Russia,Scotland,FIFA World Cup,Málaga,Spain,True
2,1987-07-27,São Tomé and Príncipe,Angola,Friendly,São Tomé,São Tomé and Príncipe,False
3,2011-09-02,Grenada,Belize,FIFA World Cup qualification,St. George's,Grenada,False
4,1998-11-18,Luxembourg,Belgium,Friendly,Luxembourg,Luxembourg,False


In [3]:
xval=pd.read_csv("/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/X_test.csv")
ytrain=pd.read_csv("/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/y_train.csv")
ytrain.head()

,result
0,1
1,0
2,0
3,1
4,0


In [4]:
xtrain.describe()

,date,home_team,away_team,tournament,city,country,neutral
count,41660,41660,41660,41660,41660,41660,41660
unique,15184,327,320,161,2014,271,2
top,2012-02-29,Brazil,Uruguay,Friendly,Kuala Lumpur,United States,False
freq,60,547,514,16079,633,1205,30971


In [5]:
xtrain.isnull().sum()

date          0
home_team     0
away_team     0
tournament    0
city          0
country       0
neutral       0
dtype: int64

In [6]:
label_encoder = LabelEncoder()
xtrain['date']=label_encoder.fit_transform(xtrain['date'])
xtrain['home_team'] = label_encoder.fit_transform(xtrain['home_team'])
xtrain['away_team'] = label_encoder.fit_transform(xtrain['away_team'])
xtrain['tournament'] = label_encoder.fit_transform(xtrain['tournament'])
xtrain['city'] = label_encoder.fit_transform(xtrain['city'])
xtrain['country'] = label_encoder.fit_transform(xtrain['country'])
xtrain['neutral'] = label_encoder.fit_transform(xtrain['neutral'])


In [7]:
xtrain.head()

,date,home_team,away_team,tournament,city,country,neutral
0,11186,36,187,78,1091,129,1
1,7154,232,242,74,1187,224,1
2,8182,278,9,78,1748,232,0
3,13364,115,29,75,1694,93,0
4,10641,163,28,78,1023,134,0


In [8]:
xtrain.describe()

,date,home_team,away_team,tournament,city,country,neutral
count,41660.00000,41660.000000,41660.000000,41660.000000,41660.000000,41660.000000,41660.000000
mean,9714.85144,158.726308,163.059193,73.326980,948.624076,139.204681,0.256577
std,4233.84454,90.677020,89.418682,36.773374,582.089995,77.369825,0.436749
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6516.00000,85.000000,88.000000,74.000000,426.000000,76.000000,0.000000
50%,10757.00000,151.000000,160.000000,78.000000,945.500000,135.000000,0.000000
75%,13367.00000,232.000000,242.000000,78.000000,1454.000000,213.000000,1.000000
max,15183.00000,326.000000,319.000000,160.000000,2013.000000,270.000000,1.000000


In [9]:
xval.head()

,date,home_team,away_team,tournament,city,country,neutral
0,2023-09-06,Macau,Bhutan,Friendly,Taipa,Macau,False
1,1966-08-13,Malaysia,Vietnam Republic,Merdeka Tournament,Kuala Lumpur,Malaysia,False
2,1951-09-02,Suriname,Guyana,Friendly,Paramaribo,Suriname,False
3,2012-08-22,India,Syria,Nehru Cup,New Delhi,India,False
4,1973-12-15,Thailand,Myanmar,King's Cup,Bangkok,Thailand,False


In [10]:
label_encoder = LabelEncoder()
xval['date']=label_encoder.fit_transform(xval['date'])
xval['home_team'] = label_encoder.fit_transform(xval['home_team'])
xval['away_team'] = label_encoder.fit_transform(xval['away_team'])
xval['tournament'] = label_encoder.fit_transform(xval['tournament'])
xval['city'] = label_encoder.fit_transform(xval['city'])
xval['country'] = label_encoder.fit_transform(xval['country'])
xval['neutral'] = label_encoder.fit_transform(xval['neutral'])

In [11]:
xval.head()

,date,home_team,away_team,tournament,city,country,neutral
0,3364,140,28,62,806,127,0
1,621,143,260,83,411,131,0
2,323,227,103,62,612,203,0
3,2822,110,230,89,575,100,0
4,880,238,160,74,70,213,0


In [12]:
xtrain, xtest, ytrain, ytest = train_test_split(xtrain, ytrain, test_size=0.2, random_state=42)

In [13]:
params = {
    'n_estimators': 180,
    'max_depth': 7,
    'learning_rate': 0.125,
    'subsample': 0.7,
    'colsample_bytree': 0.8,
    'gamma':0 ,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'objective': 'multi: ReLU', 
    'num_class': 3,  
    'eval_metric': 'merror'  
}

clf = XGBClassifier(**params)
clf.fit(xtrain,ytrain)
predictions=clf.predict(xtest)
accuracy = accuracy_score(ytest, predictions)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.5428468554968795


In [14]:

predictions = clf.predict(xtest)

# Create a DataFrame with row_id and predictions
output_df = pd.DataFrame({'row_id': range(len(predictions)), 'result': predictions})

# Save the DataFrame to a CSV file without including the index column
output_df.to_csv('predictions.csv', index=False)

print("Predictions saved to predictions.csv")
pred=pd.read_csv("/kaggle/working/predictions.csv")
pred

Predictions saved to predictions.csv


,row_id,result
0,0,2
1,1,2
2,2,2
3,3,2
4,4,2
...,...,...
8327,8327,2
8328,8328,2
8329,8329,2
8330,8330,1
